## Nicole's updates
+ cleaned the data
+ preprocessed by creating ratings matrix
+ Implemented ALS with user and item bias(fast matrix solutions)
+ temporality added

In [3]:
import pandas as pd
import numpy as np
#read in the data
data = pd.read_csv("../data/ml-latest-small/ratings.csv")

#get the necessary dimensions
n_users = data['userId'].nunique()
n_movies = data['movieId'].nunique()

In [4]:
#split the data into different bins for the temporal function
def get_bin(num):
    #subtracts 25 because they started collecting the data 25 years after january 1970
    return int((num)/(60*60*24*365)) - 25
data["bin"] = data["timestamp"].apply(get_bin)
data_groups = data.groupby("bin")

In [115]:
#create R matrix
n_users = data.userId.unique().shape[0]
n_items = data.movieId.unique().shape[0]
movieIds = sorted(data.movieId.unique())

R_init = np.zeros((n_users, n_items))
bin_match = np.zeros((n_users, n_items)) 

#stored the bin number for each user movie pair
for row in data.itertuples():
    R_init[row[1]-1, movieIds.index(row[2])] = row[3]
    bin_match[row[1]-1, movieIds.index(row[2])] = row[5]-1

In [6]:
#get initial B_i_bin_t dimensions: n_bins by n_movies
B_i_bint = np.zeros((data['bin'].nunique(),n_movies))
for i in range(len(B_i_bint)):
    sub_frame = data[data.bin==i+1]
    #for movie in movieId, then access 
    count = 0
    total = 0
    for j in range(len(movieIds)):
        r = sub_frame['rating'][sub_frame['movieId']==movieIds[j]].values
        B_i_bint[i,j] = (r.sum())/((r!=0).sum()).astype(float)
B_i_bint  = B_i_bint - (R_init.sum()/(R_init!=0).sum().astype(float))
B_i_bint[np.isnan(B_i_bint)] = 0   
    

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


In [14]:
def get_error(R, R_hat):
    #only for the that had ratings in the training matrix
    mask = (R>0)
    R_hat[mask] = 0 #set the values we don't need predictions for to 0 
    return np.sqrt(((R - R_hat)**2).mean()) #return the RMSE

def predict(U, M, mu,b_i, b_u, bin_match ,B_i_bint):
    R = np.zeros((n_users, n_movies))
    #do we need to iterate over time as well
    for u in range(n_users):#iterate over users
        for i in range(n_movies):#iterate over movies
            bin_num = int(bin_match[u,i]) #get the bin number for user u and movie i
            R[u,i] = mu+b_u[u]+b_i[i]+B_i_bint[bin_num,i]+np.dot(U[u,:],M[i,:].T)
    return R

In [119]:
#alternate implementation from online resource
np.random.seed(1)
def train(Mat, bin_match, f, lambda_, n_iter, B_i_bint):
    R = Mat[:]
    # Step 1 Initialize matrix M by assigning the average rating for that movie as the first row, and small random numbers for the remaining entries.
    M = np.random.rand(n_movies,f)
    M[:,0] = R.sum(0)/(R!=0).sum(0).astype(float)
    
    U = np.random.rand(n_users,f)#initialize U matrix
    
    mu = R.sum()/(R!=0).sum().astype(float)#average rating
    b_i = np.random.rand(n_movies) - mu #initialize b_i = len(movies)
    b_u = np.random.rand(n_users) - mu #initialize b_u = len(users)
    
    #function to minimize: sum(r_ui - ^r_ui) + lambda(b_i^2 + b_u^2+ |q|^2 + |p|^2 )
    for epoch in range(n_iter):
        #Step 2 Fix M, Solve U by minimizing the objective function
        for i in range(n_users):
            I_i = np.nonzero(R[i,:])[0] #set of movies that user i rated
            n_ui = len(I_i) #number of ratings user i has given 
            E = np.identity(f+1)
            r_u = R[i,:] - b_i - mu
            m = np.ones((n_movies,1))
            M_p = np.concatenate((m,M),1)
            M_Ii = M_p[I_i,:]
            u_prime = np.linalg.solve((np.dot(M_Ii.T,M_Ii)+(lambda_*E)), np.dot(r_u[I_i],M_Ii))
            b_u[i] = u_prime[0]
            U[i,:] = u_prime[1:]
           
        #Step 3 Fix U, solve M by minimizing the objective function similarly; 
        for j in range(n_movies):
            I_j = np.nonzero(R[:,j])[0] #set of users that rated movie j
            n_mj = len(I_j) #number of users that rated movie j
            #M_Ii = M[I_i,:] #denotes the sub-matrix of M where rows j in I_i are selected
            r_j = R[:,j] - b_u - mu
            u = np.ones((n_users,1))
            U_p = np.concatenate((u,U),1)
            U_Ij = U_p[I_j,:]
            m_prime = np.linalg.solve((np.dot(U_Ij.T,U_Ij)+(lambda_*E)), np.dot(r_j[I_j],U_Ij))
            b_i[j] = m_prime[0]
            M[j,:] = m_prime[1:]
            
        for t in range(len(B_i_bint)):
            mask = bin_match == t
            for i in range(n_movies):
                r_it = R[:,i][mask[:,i]]- mu - b_u[mask[:,i]] - b_i[i]- np.dot(U[mask[:,i]], M[i,:])
                B_i_bint[t,i] = np.sum(r_it)/(len(r_it)+lambda_)                                                   
    
        R_hat = predict(U, M, mu,b_i, b_u, bin_match ,B_i_bint)
        error = get_error(R, R_hat)
        print("iteration: ", epoch, "; error: ", error)
    
    return U, M,mu, b_u, b_i, B_i_bint
        
#attempted implementation according to the paper. Runs into singulatity problems

In [120]:
U_hat, M_hat, mu, b_u_hat, b_i_hat, B_i_bint_hat = train(R_init, bin_match, 50, 0.4, 10, B_i_bint)

iteration:  0 ; error:  0.4813180482744734
iteration:  1 ; error:  0.48141074806865386
iteration:  2 ; error:  0.48139604862382757
iteration:  3 ; error:  0.4813807462506248
iteration:  4 ; error:  0.4813632555575
iteration:  5 ; error:  0.4813495485294473
iteration:  6 ; error:  0.4813421329178382
iteration:  7 ; error:  0.4813395171944816
iteration:  8 ; error:  0.48134035135433345
iteration:  9 ; error:  0.48134393177001916


In [ ]:
def cv(dat_train, K, f, lambda_):
    